In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

while 'notebooks' in os.getcwd():
    os.chdir('..')

import torch
import torch.nn.functional as F
from ogb.nodeproppred import DglNodePropPredDataset, Evaluator
from sklearn.metrics import roc_auc_score
import logging
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import numpy as np

from src.dgl_models import GraphSAGE
from src.data.node_classifier.arxiv import load_dataset, get_symmetric_graph
from src.train.node_classifier import WeigthedGraphSageNodeClassifier

In [3]:
logging.basicConfig(
    format='%(asctime)s - %(levelname)s : %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

In [4]:
torch.cuda.is_available()

True

In [5]:
device = f'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
device

device(type='cuda', index=0)

## Data Loading

In [6]:
dataset = load_dataset()

In [7]:
graph = get_symmetric_graph(dataset)

In [8]:
graph.number_of_edges()

2332486

In [9]:
torch.mean(graph.out_degrees() * 1.)

tensor(13.7737)

In [10]:
torch.mean(graph.in_degrees() * 1.)

tensor(13.7737)

In [11]:
split_idx = dataset.get_idx_split()

In [12]:
split_idx

{'train': tensor([     0,      1,      2,  ..., 169145, 169148, 169251]),
 'valid': tensor([   349,    357,    366,  ..., 169185, 169261, 169296]),
 'test': tensor([   346,    398,    451,  ..., 169340, 169341, 169342])}

In [13]:
features = graph.ndata['feat'].cuda()
labels = dataset.labels.cuda()
train_mask = split_idx['train'].cuda()
val_mask = split_idx['valid'].cuda()
test_mask = split_idx['test'].cuda()

In [14]:
train_nid = train_mask.nonzero().squeeze()
val_nid = val_mask.nonzero().squeeze()
test_nid = test_mask.nonzero().squeeze()

In [15]:
n_edges = graph.number_of_edges()

In [16]:
graph

Graph(num_nodes=169343, num_edges=2332486,
      ndata_schemes={'year': Scheme(shape=(1,), dtype=torch.int64), 'feat': Scheme(shape=(128,), dtype=torch.float32)}
      edata_schemes={})

In [17]:
torch.version.cuda

'11.3'

In [18]:
graph = graph.to(device)

In [19]:
n_layers = 3
n_iters = 3000
epochs = 5000
log_steps = 100
input_dim = features.shape[1]
hidden_channels = input_dim * 2
output_dim = dataset.num_classes
lr_rate = 0.001
dropout = 0.5

In [20]:
model = GraphSAGE(
    n_layers=n_layers,
    in_channels=input_dim,
    hidden_channels=hidden_channels,
    out_channels=output_dim,
    dropout=dropout)\
    .to(device)

In [21]:
def train(model, graph, features, train_mask, optimizer, edge_weight=None):
    model.train()

    optimizer.zero_grad()
    out = model(graph, features, edge_weight=edge_weight)[train_mask]
    loss = F.nll_loss(out, labels.squeeze(1)[train_mask])
    loss.backward()
    optimizer.step()

    return loss.item()

In [22]:
@torch.no_grad()
def test(model, graph, features, labels, train_mask, val_mask, test_mask, evaluator, edge_weight=None):
    model.eval()

    out = model(graph, features, edge_weight=edge_weight)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': labels[train_mask],
        'y_pred': y_pred[train_mask],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': labels[val_mask],
        'y_pred': y_pred[val_mask],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': labels[test_mask],
        'y_pred': y_pred[test_mask],
    })['acc']

    return train_acc, valid_acc, test_acc

In [23]:
evaluator = Evaluator(name='ogbn-arxiv')

In [24]:
edge_weights = None

In [25]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr_rate)
for epoch in range(1, 1 + epochs):
    loss = train(model, graph, features, train_mask, optimizer, edge_weight=edge_weights)
    result = test(model, graph, features, labels, train_mask, val_mask, test_mask, evaluator, edge_weight=edge_weights)

    if epoch % log_steps == 0:
        train_acc, valid_acc, test_acc = result
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_acc:.2f}%, '
              f'Valid: {100 * valid_acc:.2f}% '
              f'Test: {100 * test_acc:.2f}%')

Epoch: 100, Loss: 1.3237, Train: 66.12%, Valid: 66.65% Test: 65.82%
Epoch: 200, Loss: 1.1231, Train: 69.88%, Valid: 69.59% Test: 68.58%
Epoch: 300, Loss: 1.0452, Train: 71.56%, Valid: 70.61% Test: 69.55%
Epoch: 400, Loss: 0.9965, Train: 72.69%, Valid: 71.08% Test: 69.96%
Epoch: 500, Loss: 0.9572, Train: 73.43%, Valid: 71.38% Test: 70.20%
Epoch: 600, Loss: 0.9321, Train: 74.17%, Valid: 71.72% Test: 70.92%
Epoch: 700, Loss: 0.9073, Train: 74.82%, Valid: 71.83% Test: 70.95%
Epoch: 800, Loss: 0.8832, Train: 75.38%, Valid: 71.97% Test: 71.11%
Epoch: 900, Loss: 0.8636, Train: 75.98%, Valid: 72.25% Test: 71.21%
Epoch: 1000, Loss: 0.8453, Train: 76.41%, Valid: 72.27% Test: 71.04%
Epoch: 1100, Loss: 0.8335, Train: 76.90%, Valid: 72.45% Test: 71.36%
Epoch: 1200, Loss: 0.8208, Train: 77.21%, Valid: 72.47% Test: 71.48%
Epoch: 1300, Loss: 0.8072, Train: 77.70%, Valid: 72.62% Test: 71.31%
Epoch: 1400, Loss: 0.7934, Train: 78.08%, Valid: 72.70% Test: 71.44%
Epoch: 1500, Loss: 0.7798, Train: 78.48%, V